In [1]:
import Augmentor as aug
import glob
import os
import numpy as np
import cv2 as cv
import PIL
from Augmentor.Operations import Operation

In [2]:
class Lightning(Operation):
    def __init__(self, probability, intensity_low=0.7, intensity_high=1.2):
        Operation.__init__(self, probability)
        self.intensity_low = intensity_low
        self.intensity_high = intensity_high

    def perform_operation(self, images):
        for i, image in enumerate(images):
            image = np.array(image.convert('RGB'))
            row, col, _ = image.shape
            light_intensity = np.random.randint(
                int(self.intensity_low * 100),
                int(self.intensity_high * 100)
            )
            light_intensity /= 100

            gaussian = 100 * np.random.random((row, col, 1))
            gaussian = np.array(gaussian, dtype=np.uint8)
            gaussian = np.concatenate((gaussian, gaussian, gaussian), axis=2)

            image = cv.addWeighted(image, light_intensity, gaussian, 0.25, 0)

            image = PIL.Image.fromarray(image)
            images[i] = image
            
        return images

In [3]:
DIRECTORIES = 'dataset/*'
folders = []
for f in glob.glob(DIRECTORIES):
    if os.path.isdir(f):
        folders.append(os.path.abspath(f))

print('Classes found {}'.format([os.path.split(x)[1] for x in folders]))


Classes found ['Wart', 'Melanocytic naevus', 'Lentigo', 'Pyogenic Granuloma', 'Haemangioma', 'Squamous Cell Carcinoma', 'Basal Cell Carcinoma', 'Dermatofibroma', 'seborrhoeic keratosis', 'Malignant Melanoma', 'Actinic Keratosis', 'Intraepithelial Carcinoma']


In [4]:
pipelines = {}
for folder in folders:
    pipelines[os.path.split(folder)[1]] = \
        (aug.Pipeline(source_directory=folder,
            output_directory='resnet_augmented',
            save_format='jpg'
            )
        )

classes_count = []
for p in pipelines.values():
    if len(p.augmentor_images) > 0:
        print("Class '{}' has {} samples".format(
            p.augmentor_images[0].class_label,
            len(p.augmentor_images)
        ))
        classes_count.append(len(p.augmentor_images))

Initialised with 102 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Wart/resnet_augmented.Initialised with 332 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Melanocytic naevus/resnet_augmented.Initialised with 81 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Lentigo/resnet_augmented.Initialised with 24 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Pyogenic Granuloma/resnet_augmented.Initialised with 115 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Haemangioma/resnet_augmented.Initialised with 328 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Squamous Cell Carcinoma/resnet_augmented.Initialised with 720 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Basal Cell Carcinoma/resnet_augmented.Initialised with 106 image(s) found.
Output directory set to 

In [5]:
lightning = Lightning(probability=0.5)

for p in pipelines.values():
    p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
    p.zoom_random(probability=0.4, percentage_area=0.9)
    p.flip_left_right(probability=0.7)
    p.flip_top_bottom(probability=0.5)
    p.random_distortion(probability=0.8, grid_width=5, grid_height=5, magnitude=15)
    p.add_operation(lightning)
    p.resize(probability=1.0, width=224, height=224)

In [6]:
SAME_SAMPLING = False
MULTIPLIER = 29

for p in pipelines.values():
    if SAME_SAMPLING:
        diff = max(classes_count) - len(p.augmentor_images)
        p.sample((len(p.augmentor_images) + diff)*MULTIPLIER + diff)
    else:
        p.sample(len(p.augmentor_images)*MULTIPLIER)

Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7FC10374C700>:   0%|          | 10/2958 [00:00<00:49, 59.00 Samples/s]dict_values([<Augmentor.Pipeline.Pipeline object at 0x7fc1039aaa00>, <Augmentor.Pipeline.Pipeline object at 0x7fc1038d3e80>, <Augmentor.Pipeline.Pipeline object at 0x7fc10390ca00>, <Augmentor.Pipeline.Pipeline object at 0x7fc1038a58b0>, <Augmentor.Pipeline.Pipeline object at 0x7fc1038a1070>, <Augmentor.Pipeline.Pipeline object at 0x7fc14f7ce7c0>, <Augmentor.Pipeline.Pipeline object at 0x7fc1038d0430>, <Augmentor.Pipeline.Pipeline object at 0x7fc1037f1e20>, <Augmentor.Pipeline.Pipeline object at 0x7fc103804310>, <Augmentor.Pipeline.Pipeline object at 0x7fc1037b1280>, <Augmentor.Pipeline.Pipeline object at 0x7fc1038794f0>, <Augmentor.Pipeline.Pipeline object at 0x7fc1037bec40>])
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7FC0E9FE9070>: 100%|██████████| 2958/2958 [00:13<00:00, 220.83 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size